In [0]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [0]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

0it [00:00, ?it/s]

100%|█████████▉| 170024960/170498071 [00:17<00:00, 6530107.23it/s]

Files already downloaded and verified


In [0]:
train_loader = DataLoader(dataset=dataset_train, shuffle=True)
test_loader = DataLoader(dataset=dataset_test, shuffle=False)

In [0]:
# Definir a arquitetura MLP

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32*32, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3= nn.Linear(50, 10)
        #self.activation_function = nn.Sigmoid()
        self.activation_function = nn.ReLU()
        self.soft_max = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        x = self.activation_function(self.fc3(x))
        x = self.soft_max(x)
        return x

In [0]:
model = MLP()
print(model)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = model.to(device)

MLP(
  (fc1): Linear(in_features=1024, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (activation_function): ReLU()
  (soft_max): Softmax()
)
cuda:0


In [0]:
# Definir otimizador e loss
# Nota: testar outros otimizadores e funções de loss (em particular cross entropy)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
loss_fn = torch.nn.MSELoss().to(device)

In [0]:
# Avaliar o modelo aqui (no conjunto de teste)
def validate():
  
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
      
        for data in test_loader:
          
            images, labels = data
            
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return correct / total

In [0]:
from tqdm.auto import tqdm

# Realizar o treinamento aqui

epochs = 100

model.train()


for epoch in range(epochs):
  
  avg_loss = 0
  hits = 0
  total = 0
  #progress_bar = tqdm(train_loader)
  
  for i, (X,y) in enumerate(train_loader):
      model.train()
      #progress_bar.set_description("Epoch " + str(epoch))
      
      # Send to device
      X = X.to(device)
      y = y.to(device)
      # convert label to one-hot tensor
      onehot_y = np.zeros(10)
      onehot_y[y.item()] = 1
      onehot_y = torch.cuda.FloatTensor(onehot_y)    
      
      optimizer.zero_grad()
      
      # get output of model
      output = model(X)
      
      
      # calculate loss and backpropagates
      loss = loss_fn(output, onehot_y)
      loss.backward()
      optimizer.step()
      
      avg_loss += loss.item()
      
      _, pred = torch.max(output.data,1)
      hits += (pred == y).sum().item()
      total += y.size(0)
      train_accuracy = hits / total
      
      #progress_bar.set_postfix(xentropy='%.3f' % (avg_loss / (i + 1)),
      #                 acc='%.3f' % (train_accuracy))

#      if i % 10000 == 9999:
#        print("[", epoch, ",", i+1,"] " "loss: ", avg_loss / 10000)
#        avg_loss = 0

  test_accuracy = validate()
  
  #tqdm.write('test_acc: %.3f' % (test_accuracy))
  print("[", epoch, "]", "test_acc: ", (test_accuracy), "train_acc:", (train_accuracy), "loss:", avg_loss / len(train_loader))

170500096it [00:29, 6530107.23it/s]                               

[ 0 ] test_acc:  0.2253 train_acc: 0.21354 loss: 0.08705977026007924
[ 1 ] test_acc:  0.3176 train_acc: 0.29244 loss: 0.08245192813069989
[ 2 ] test_acc:  0.3345 train_acc: 0.32396 loss: 0.07990337094082026
[ 3 ] test_acc:  0.3439 train_acc: 0.33992 loss: 0.07860952974329652
[ 4 ] test_acc:  0.3535 train_acc: 0.35164 loss: 0.07763421745451388
[ 5 ] test_acc:  0.3589 train_acc: 0.3603 loss: 0.07692509121911138
[ 6 ] test_acc:  0.3568 train_acc: 0.36832 loss: 0.07615795637980656
[ 7 ] test_acc:  0.3755 train_acc: 0.37604 loss: 0.0755592403987661
[ 8 ] test_acc:  0.3651 train_acc: 0.38222 loss: 0.07502730949485885
[ 9 ] test_acc:  0.3648 train_acc: 0.38672 loss: 0.07449655624136574
[ 10 ] test_acc:  0.3736 train_acc: 0.39532 loss: 0.07398176760308943
[ 11 ] test_acc:  0.3856 train_acc: 0.39738 loss: 0.07367780449467844
[ 12 ] test_acc:  0.3728 train_acc: 0.40082 loss: 0.07319603488063563
[ 13 ] test_acc:  0.3643 train_acc: 0.4066 loss: 0.07282234460966923
[ 14 ] test_acc:  0.3886 train_ac